**Script for Challenge 3**
**Authors : Aditya Lahiri, Kanchan Satpute and Harish Kumar **

In [0]:
import pandas as pd
from google.colab import files 
import numpy as np

**Load the training set**

In [4]:
uploaded_train = files.upload()


Saving population_training.csv to population_training.csv


**Load the testing set**

In [5]:
uploaded_test = files.upload()


Saving population_testing.csv to population_testing.csv


In [6]:
for fn1 in uploaded_train.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn1, length=len(uploaded_train[fn1])))
  
for fn2 in uploaded_test.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn2, length=len(uploaded_test[fn2])))

User uploaded file "population_training.csv" with length 89206 bytes
User uploaded file "population_testing.csv" with length 41027 bytes


**Converting the training data into a dataframe**

In [0]:
import io 
population_training_df = pd.read_csv(io.StringIO(uploaded_train['population_training.csv'].decode('cp1252'))).dropna(axis=0)
population_training_df.head(150) 


**Converting the testing data into a dataframe**

In [0]:
population_testing_df= pd.read_csv(io.StringIO(uploaded_test['population_testing.csv'].decode('cp1252'))).dropna(axis=0)
population_testing_df=population_testing_df.drop(population_testing_df.index[123])
population_testing_df.head(150)


**Find the size of  test data **

In [0]:
print(population_testing_df.shape)


**Remove the country name column**

In [0]:
population_training_df.drop(['Country Name'], axis=1, inplace=True)
population_testing_df.drop(['Country Name'], axis=1, inplace=True)


**Create a training and testing set**

In [0]:
population_training_matrix = population_training_df.values
population_testing_matrix = population_testing_df.values

**print the shape of training matrix**

In [0]:
print(population_training_matrix.shape)
population_training_matrix

**print the shape of testing matrix**

In [0]:
print(population_testing_matrix.shape)
population_testing_matrix

In [0]:
population_training_matrix.transpose()

**Build the regressors**

In [23]:
alphas = 10 ** (np.linspace(0.001, 10, 4000)) # 10 ,500 # 100, 8000 # 10, 4000 #
#alphas=10**np.linspace(11,-2,500)*0.5
#alphas=10 ** (np.linspace(1, 17, 2000)) good
#alphas=10 ** (np.linspace(0.01, 25, 4000))
print(alphas.min())
print(alphas.max())

1.0023052380778996
10000000000.0


In [0]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import Lasso
Y_Predict_Matrix = np.zeros((258,17))
Coeff_Matrix= np.zeros((258,257))#257,256
Number_Country_More=0
for iter_lasso in range(0,258):
  mse_list=[]
  alpha_list=[]
  for iter_alphas in alphas:
    lasso = Lasso(normalize="True",tol=0.01,max_iter=500000)
    lasso.set_params(alpha = iter_alphas)
    List_Ind = list(range(0,258))
    List_Ind.remove(iter_lasso)
    lasso.fit(population_training_matrix.transpose()[:,List_Ind],population_training_matrix.transpose()[:,iter_lasso])
    Predicted_Values=lasso.predict(population_training_matrix.transpose()[:,List_Ind])#training not testing
    
    if (0<np.count_nonzero(lasso.coef_)<=5):
      mse_list.append(mse(population_training_matrix.transpose()[:,iter_lasso],Predicted_Values))# training
      alpha_list.append(iter_alphas)
      
  if(len(alpha_list)==0):
    lasso = Lasso(normalize="True",tol=0.01,max_iter=500000)
    lasso.set_params(alpha = alphas.max())
    lasso.fit(population_training_matrix.transpose()[:,List_Ind],population_training_matrix.transpose()[:,iter_lasso])
    Y_Predict_Matrix[iter_lasso,:]=lasso.predict(population_testing_matrix.transpose()[:,List_Ind])
    Coeff_Matrix[iter_lasso,:]=lasso.coef_
    Number_Country_More=Number_Country_More+1
  
  elif(len(alpha_list)>0):
    select_alpha_index=np.argmin(mse_list)
    aplha_selected=alpha_list[select_alpha_index]
    lasso = Lasso(normalize="True",tol=0.01,max_iter=500000)
    lasso.set_params(alpha = aplha_selected)
    lasso.fit(population_training_matrix.transpose()[:,List_Ind],population_training_matrix.transpose()[:,iter_lasso])
    Y_Predict_Matrix[iter_lasso,:]=lasso.predict(population_testing_matrix.transpose()[:,List_Ind])
    Coeff_Matrix[iter_lasso,:]=lasso.coef_
    
    

       

       
       
    
  


      
    

    



  



In [18]:
Number_Country_More

0

In [19]:
print(np.shape(Y_Predict_Matrix))
print(np.shape(Coeff_Matrix))
print(Number_Country_More)
print(Coeff_Matrix)
Coef_Mat_Sq=np.zeros((258,258))
for iter_row in range(0,258):
  elem_one=0
  for iter_col in range(0,258):
    if(iter_row==iter_col):
      Coef_Mat_Sq[iter_row,iter_col]=0
    else:
      Coef_Mat_Sq[iter_row,iter_col]=Coeff_Matrix[iter_row,elem_one]
      elem_one=elem_one+1
print(np.shape(Coef_Mat_Sq))



      

  

(258, 17)
(258, 257)
0
[[2.31889301e-03 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.78142742e+02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(258, 258)


In [20]:
from decimal import Decimal

print(np.shape(Y_Predict_Matrix))
print(np.shape(population_testing_matrix))
error_1=0
for iter_error in range (0,258):
  error_1 = mse(Y_Predict_Matrix[iter_error,:],population_testing_matrix[iter_error,:])
  error_1=error_1+error_1
print(error_1)
'%.2E' % Decimal(error_1)

(258, 17)
(258, 17)
48384561840582.19


'4.84E+13'

In [0]:
np.savetxt('3challenge.csv',Y_Predict_Matrix.transpose(),delimiter=",")
files.download('3challenge.csv')

In [0]:
np.savetxt('population_parameters.csv',Coef_Mat_Sq,delimiter=",")
files.download('population_parameters.csv')